In [11]:
import random
import matplotlib.pyplot as plt
import time

# **Criptografía RSA: De la Matemática al Código**


Este notebook explica el funcionamiento del algoritmo RSA, desde sus cimientos matemáticos hasta la implementación de un ataque de factorización de Fermat.

## 1. Fundamentos Matemáticos

Antes de implementar RSA, necesitamos algunas herramientas de teoría de números.

**1.1 Exponenciación Modular Rápida.**

RSA requiere elevar números grandes a potencias muy grandes. Calcular $a^b \pmod n$ de forma directa (multiplicando $a$ por sí mismo $b$ veces) es computacionalmente inviable cuando $b$ tiene cientos de dígitos. La Exponenciación Binaria (o método de Square-and-Multiply) aprovecha las propiedades de los exponentes para reducir drásticamente el número de operaciones.

El algoritmo se basa en dos identidades clave de la aritmética modular:
- Si el exponente es par: $a^{2k} = (a^k)^2$. 
- Si el exponente es impar: $a^{2k+1} = a \cdot (a^k)^2$.
- Propiedad Distributiva: $(a \cdot b) \pmod n = [(a \pmod n) \cdot (b \pmod n)] \pmod n$.

Para calcular $a^b \pmod n$, el algoritmo descompone el exponente $b$ en su representación binaria. Sea $b$ un número de $k$ bits:$$b = (b_{k-1}b_{k-2}\dots b_1b_0)_2 = \sum_{i=0}^{k-1} b_i \cdot 2^i$$Donde cada $b_i$ es $0$ o $1$. Sustituyendo esto en la potencia, obtenemos:$$a^b = a^{\sum b_i 2^i} = a^{b_0 2^0} \cdot a^{b_1 2^1} \cdot a^{b_2 2^2} \dots a^{b_{k-1} 2^{k-1}}$$

Esta descomposición matemática se traduce en un algoritmo que realiza únicamente dos operaciones por bit, manteniendo los números siempre dentro de un rango manejable gracias a la propiedad distributiva del módulo:
- Cuadrado de la base con reducción: En cada paso $i$, calculamos la potencia de base 2: $a^{2^i} \equiv (a^{2^{i-1}})^2 \pmod n$. El uso del módulo en este paso es crítico: evita que la base crezca exponencialmente en tamaño de bits, manteniéndola siempre menor a $n$.
- Multiplicación condicional: Si el bit $b_i$ es $1$, el acumulador se actualiza: $R \equiv (R \cdot a^{2^i}) \pmod n$.

In [ ]:
def exponenciacion_modular(a, b, n):
    resultado = 1
    a = a % n
    # El bucle continúa hasta procesar todos los bits del exponente.
    while b > 0:
        if b & 1:       # Si el bit actual es 1, se multiplica el resultado.
            resultado = (resultado * a) % n
        a = (a * a) % n # La base se eleva al cuadrado en cada paso.
        b >>= 1         # Desplazamiento de bit para procesar el exponente

    return resultado

**1.2 Algoritmo de Euclides Extendido e Inverso Modular** 

Para encontrar la clave privada $d$, necesitamos resolver la ecuación:$$e \cdot d \equiv 1 \pmod{\phi(n)}$$Esto se logra mediante el Algoritmo de Euclides Extendido, que encuentra los coeficientes $x, y$ tales que $ax + by = \text{mcd}(a, b)$

In [13]:
def mcd_extendido(a, b):
    if a == 0:
        return b, 0, 1
    g, x1, y1 = mcd_extendido(b % a, a)
    return g, y1 - (b // a) * x1, x1

def inverso_modular(a, m):
    g, x, _ = mcd_extendido(a, m)
    if g != 1:
        raise Exception("No existe inverso modular")
    return x % m


## 2. Test de Primalidad: Miller-Rabin

RSA depende de encontrar números primos gigantes. Como no hay una fórmula para generar primos, generamos un número aleatorio y verificamos si es primo usando el test probabilístico de Miller-Rabin.
Si un número pasa 40 rondas de este test, la probabilidad de que sea un falso positivo (compuesto) es menor a $2^{-80}$, lo cual es aceptable para estándares de seguridad.


In [20]:
def ronda_miller_rabin(n, a):
    # n - 1 = d * 2^s con d impar.
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    
    x = exponenciacion_modular(a, d, n)
    if x == 1 or x == n - 1:
        return True
    for _ in range(s - 1):
        x = (x * x) % n
        if x == n - 1:
            return True
    return False

def es_primo_probable(n, rondas=40):
    if n < 2: return False
    if n in (2, 3): return True
    if n % 2 == 0: return False
    for _ in range(rondas):
        a = random.randrange(2, n - 1)
        if not ronda_miller_rabin(n, a):
            return False
    return True

def generar_primo(bits):
    while True:
        n = random.getrandbits(bits)
        n |= (1 << (bits - 1)) | 1 # Asegura tamaño y que sea impar
        if es_primo_probable(n):
            return n


## 3. El Algoritmo RSA

El proceso se resume en:
1. Generar claves: 
    - Elegir dos primos $p$ y $q$.
    - $n = p \cdot q$.
    - $\phi(n) = (p-1)(q-1)$.
    - Elegir $e$ (comúnmente $65537$).
    - Calcular $d$ como el inverso de $e$ módulo $\phi(n)$.
2. Cifrar: $C = M^e \pmod n$.
3. Descifrar: $M = C^d \pmod n$.


In [15]:
def generar_claves_rsa(bits=1024):
    p = generar_primo(bits)
    q = generar_primo(bits)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = inverso_modular(e, phi)
    return (n, e), (n, d)

def cifrar_rsa(mensaje, clave_publica):
    n, e = clave_publica
    return exponenciacion_modular(mensaje, e, n)

def descifrar_rsa(cifrado, clave_privada):
    n, d = clave_privada
    return exponenciacion_modular(cifrado, d, n)

## 4. Vulnerabilidades: El Ataque de Fermat

La seguridad de RSA reside en la dificultad de factorizar $n$. Sin embargo, si $p$ y $q$ están muy cerca uno del otro, $n$ puede factorizarse rápidamente.

El ataque de Fermat se basa en que cualquier número impar puede representarse como una diferencia de cuadrados:$$n = x^2 - y^2 = (x - y)(x + y)$$Si $p \approx q$, entonces $x = \frac{p+q}{2}$ estará muy cerca de $\sqrt{n}$.

In [16]:
def raiz_entera(n):
    if n == 0: return 0
    x = 2**(n.bit_length() // 2 + 1)
    while True:
        y = (x + n // x) // 2
        if y >= x: return x
        x = y

def ataque_de_fermat(n, K=10000000):
    x = raiz_entera(n)
    if x * x < n: x += 1
    for _ in range(K):
        y2 = x*x - n
        y = raiz_entera(y2)
        if y*y == y2:
            return (x - y), (x + y)
        x += 1
    return None

## 5. Simulación de Comunicación y Criptoanálisis

En esta sección, simularemos un flujo de comunicación real. Dividiremos el proceso en tres etapas: la generación y envío legítimo, la intercepción del mensaje y el intento de quiebre de la clave.

**5.1 Caso 1: Flujo de Comunicación Seguro**

En este escenario, el *Receptor* genera sus claves utilizando primos aleatorios y suficientemente distantes. El *Emisor* cifra un mensaje y lo envía.


In [23]:
# Funciones auxiliares de conversión texto-entero y viceversa.
def texto_a_entero(texto):
    data = texto.encode("utf-8")
    return int.from_bytes(data, "big")

def entero_a_texto(entero):
    data = entero.to_bytes((entero.bit_length() + 7) // 8, "big")
    return data.decode("utf-8")

# 1. Configuración del Receptor (Generación de Claves)
# 1024 bits para asegurar una complejidad computacional adecuada.
publica_rec, privada_rec = generar_claves_rsa(1024)

# 2. Acción del Emisor (Cifrado)
mensaje_original = "Información confidencial: El presupuesto ha sido aprobado."
print(f"Mensaje Original: {mensaje_original}")

msj_entero = texto_a_entero(mensaje_original)
mensaje_cifrado = cifrar_rsa(msj_entero, publica_rec)

print(f"\nMensaje cifrado enviado al canal: \n{mensaje_cifrado}")

# 3. Recepción y Descifrado
msj_final_entero = descifrar_rsa(mensaje_cifrado, privada_rec)
print(f"\nMensaje descifrado por el Receptor: {entero_a_texto(msj_final_entero)}")

Mensaje Original: Información confidencial: El presupuesto ha sido aprobado.

Mensaje cifrado enviado al canal: 
16179268997876182006541304712972728073693653119651666230838649445796992146052863825295394773726888006367808502669063010483252271548012891958766781977291269497261676175462723680545242365285496140358187403933438657439383754716503984356485774600042053517851382364260973677359843116039839379116472909814762621416173472027866336634951219947609162695617794281671931221020833526040317659816581544424064393480961726761002733416436155865062123585471470097591444371297058137277709979878519477954740709873482904987774677903323058254994243035830653260998152925459346054110952005732406510869619112717710086525204641696008031270710

Mensaje descifrado por el Receptor: Información confidencial: El presupuesto ha sido aprobado.


**5.2 Caso 2: Intercepción y Ataque (Criptoanálisis de Fermat)**


Aquí simularemos una vulnerabilidad crítica: el *Receptor* utiliza un software de generación de claves defectuoso que elige dos números primos muy cercanos entre sí. El *Interceptor* captura el tráfico y aplica el algoritmo de Fermat.

In [24]:
print("--- INICIO DE INTERCEPCIÓN ---")

# 1. Generación de clave vulnerable (Primos cercanos)
p_vulnerable = generar_primo(1024)
# Se fuerza un segundo primo muy cercano al primero
q_vulnerable = p_vulnerable + 2
while not es_primo_probable(q_vulnerable):
    q_vulnerable += 2

n_vulnerable = p_vulnerable * q_vulnerable
e = 65537
publica_vulnerable = (n_vulnerable, e)

# 2. El Emisor envía un mensaje sin saber que la clave es débil
mensaje_secreto = "Clave de acceso a la base de datos: Admin123"
cifrado_interceptado = cifrar_rsa(texto_a_entero(mensaje_secreto), publica_vulnerable)

print(f"Paquete interceptado: {str(cifrado_interceptado)[:100]}...")

# 3. El Interceptor ejecuta el ataque de Fermat
print("\n[Interceptor] Analizando vulnerabilidades en 'n'...")
inicio_ataque = time.time()

# El ataque busca x tal que x^2 - n sea un cuadrado perfecto
resultado = ataque_de_fermat(n_vulnerable, K=10**6)
fin_ataque = time.time()

if resultado:
    p_ext, q_ext = resultado
    print(f"[Interceptor] ¡Factorización exitosa!")
    print(f"Tiempo de quiebre: {fin_ataque - inicio_ataque:.4f} segundos")
    
    # Reconstrucción de la clave privada
    phi_ext = (p_ext - 1) * (q_ext - 1)
    d_ext = inverso_modular(e, phi_ext)
    privada_ext = (n_vulnerable, d_ext)
    
    # Descifrado del mensaje interceptado
    mensaje_recuperado = entero_a_texto(descifrar_rsa(cifrado_interceptado, privada_ext))
    print(f"[Interceptor] Contenido recuperado: '{mensaje_recuperado}'")
else:
    print("[Interceptor] El ataque falló. La clave no presenta la vulnerabilidad de Fermat.")


--- INICIO DE INTERCEPCIÓN ---
Paquete interceptado: 9338386306758358599837060514348318041402612489496892644279298126576485002951396244113172509466960823...

[Interceptor] Analizando vulnerabilidades en 'n'...
[Interceptor] ¡Factorización exitosa!
Tiempo de quiebre: 0.0001 segundos
[Interceptor] Contenido recuperado: 'Clave de acceso a la base de datos: Admin123'
